<a href="https://colab.research.google.com/github/FibGro/Pyspark/blob/main/Structure_Data_DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━ 256.8/317.0 MB 104.0 MB/s eta 0:00:01